# Project 4 - Based on Grid Search best params train with all data

# Step 1

Libraries

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
import torch
import torch.nn as nn

# Step 2

Database

In [3]:
np.random.seed(123)
torch.manual_seed(123)

In [11]:
forecasters = pd.read_csv('/content/entradas_breast.csv')
labels = pd.read_csv('/content/saidas_breast.csv')

In [13]:
forecasters = torch.tensor(np.array(forecasters), dtype = torch.float)
labels = torch.tensor(np.array(labels), dtype = torch.float)

In [14]:
type(forecasters)

torch.Tensor

# Step 3

Casting np to tensor

In [15]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(forecasters, labels), batch_size=10, shuffle=True)

# Step 4

Model building

In [18]:
class torch_classifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.dense0 = nn.Linear(30,8)
    torch.nn.init.normal_(self.dense0.weight, mean=0.0, std=0.05)

    self.dense1 = nn.Linear(8,8)
    torch.nn.init.normal_(self.dense1.weight, mean=0.0, std=0.05)

    self.dense2 = nn.Linear(8,1)
    self.activation = nn.ReLU()

    self.dropout = nn.Dropout(0,2)
    self.output = nn.Sigmoid()

  def forward(self, X):
    X = self.dense0(X)
    X = self.activation(X)
    X = self.dropout(X)
    X = self.dense1(X)
    X = self.activation(X)
    X = self.dropout(X)
    X = self.dense2(X)
    X = self.output(X)
    return X

In [20]:
classifier = torch_classifier()

In [22]:
criterion = nn.BCELoss()

optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001, weight_decay=0.0001)

# Step 5

Train model

In [23]:
for epoch in range(100):
  running_loss = 0.

  for data in train_loader:
    inputs, label = data
    optimizer.zero_grad()

    outputs = classifier(inputs)
    loss = criterion(outputs, label)

    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  print('Epoch: %3d: Loss %.5f' % (epoch+1, running_loss/len(train_loader)))


Epoch:   1: Loss 0.77364
Epoch:   2: Loss 0.53998
Epoch:   3: Loss 0.51572
Epoch:   4: Loss 0.47303
Epoch:   5: Loss 0.44338
Epoch:   6: Loss 0.41656
Epoch:   7: Loss 0.38030
Epoch:   8: Loss 0.35354
Epoch:   9: Loss 0.36285
Epoch:  10: Loss 0.32307
Epoch:  11: Loss 0.31945
Epoch:  12: Loss 0.29994
Epoch:  13: Loss 0.31773
Epoch:  14: Loss 0.27684
Epoch:  15: Loss 0.26860
Epoch:  16: Loss 0.25721
Epoch:  17: Loss 0.24745
Epoch:  18: Loss 0.23860
Epoch:  19: Loss 0.23120
Epoch:  20: Loss 0.22687
Epoch:  21: Loss 0.20521
Epoch:  22: Loss 0.24023
Epoch:  23: Loss 0.21920
Epoch:  24: Loss 0.27274
Epoch:  25: Loss 0.20164
Epoch:  26: Loss 0.19937
Epoch:  27: Loss 0.20380
Epoch:  28: Loss 0.19277
Epoch:  29: Loss 0.20407
Epoch:  30: Loss 0.19988
Epoch:  31: Loss 0.20875
Epoch:  32: Loss 0.19843
Epoch:  33: Loss 0.20365
Epoch:  34: Loss 0.18737
Epoch:  35: Loss 0.19361
Epoch:  36: Loss 0.18842
Epoch:  37: Loss 0.18134
Epoch:  38: Loss 0.17673
Epoch:  39: Loss 0.17300
Epoch:  40: Loss 0.18159


# Step 6

classifying only one record

In [24]:
new = torch.tensor([[15.80, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178,
                  0.20, 0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.015,
                  0.03, 0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185,
                  0.84, 158, 0.363]], dtype = torch.float)

In [25]:
classifier.eval()

torch_classifier(
  (dense0): Linear(in_features=30, out_features=8, bias=True)
  (dense1): Linear(in_features=8, out_features=8, bias=True)
  (dense2): Linear(in_features=8, out_features=1, bias=True)
  (activation): ReLU()
  (dropout): Dropout(p=0, inplace=2)
  (output): Sigmoid()
)

In [26]:
forecasting = classifier(new)

In [27]:
forecasting

tensor([[1.]], grad_fn=<SigmoidBackward0>)

In [28]:
forecasting = forecasting.detach()

In [29]:
forecasting

tensor([[1.]])

In [30]:
forecasting = forecasting.numpy()

In [31]:
type(forecasting)

numpy.ndarray

In [32]:
forecasting

array([[1.]], dtype=float32)

In [33]:
forecasting = (forecasting > 0.5)
forecasting

array([[ True]])

# Step 7

Saving classifier

In [35]:
classifier.state_dict()

OrderedDict([('dense0.weight',
              tensor([[ 1.6502e-01,  6.0314e-02,  5.0601e-01,  1.4060e-01, -3.8562e-02,
                       -1.3699e-01, -6.6027e-04,  1.2158e-01, -1.7807e-03,  1.7545e-01,
                        5.6647e-02,  5.3652e-02, -9.2483e-03, -1.1478e-02, -3.3487e-02,
                        5.3917e-02,  2.8183e-01,  3.2757e-01,  1.4797e-01, -4.9551e-01,
                        8.6561e-02, -1.5248e-01,  3.2011e-01, -1.6158e-01, -1.2600e-03,
                       -3.8763e-02, -7.6130e-02,  8.3725e-02, -4.2798e-02, -4.4904e-01],
                      [ 6.1736e-02, -2.0170e-02,  3.4046e-01,  1.3590e-02, -1.6125e-01,
                        6.8624e-03, -3.3806e-02,  2.3744e-01,  3.9160e-02, -2.0839e-01,
                       -8.8423e-02,  1.4717e-02,  3.5499e-02, -1.2219e-01, -1.6580e-02,
                        3.2146e-01,  2.0999e-01,  2.5235e-01, -3.7452e-01, -4.2356e-01,
                        1.3984e-01, -6.2029e-02,  3.1465e-01, -8.8413e-02, -7.3453e-02,


In [36]:
torch.save(classifier.state_dict(), 'classifier.pth')